In [ ]:
import keras 
import numpy as np
import pdb
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Flatten, TimeDistributed
from keras.callbacks import LambdaCallback, ModelCheckpoint

num_chars = 256
seq_len = 100
batch_size = 32
model_save_path = 'serialized_models/shakespeare_gen.h5'

def bliteral_to_categorical(b_string):
    # Convert byte literal representation
    int_rep = [ord(c) for c in b_string]
    return keras.utils.to_categorical(int_rep, num_classes=num_chars)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    random_char = chr(np.random.randint(num_chars))
    start_char = bliteral_to_categorical(''.join([random_char for i in range(seq_len)]))
    curr_model_input = start_char.reshape(1, *start_char.shape)
    for i in range(len(start_char) - 1):
        curr_index_pred = model.predict(curr_model_input)[0, i]
        model_next_input = np.zeros(num_chars)
        model_next_input[np.argmax(curr_index_pred)] = 1
        curr_model_input[0, i + 1] = model_next_input
    
    final_output = model.predict(curr_model_input)[0]
    best_pred_chars = np.argmax(final_output, axis=1)
    str_out = ''.join([chr(r) for r in best_pred_chars])
    print('Generated string: {}'.format(str_out))
    
def generate_sequences(inputs, targets, seq_len, batch_size):
    while True:
        seq_starts = np.random.choice(len(inputs) - seq_len, batch_size)
        X_batch = [inputs[s:s+seq_len] for s in seq_starts]
        Y_batch = [targets[s:s+seq_len] for s in seq_starts]
        yield np.array(X_batch), np.array(Y_batch)

with open('data/hamlet.txt', 'r') as f:
    model = Sequential([
        LSTM(15, input_shape=(seq_len, num_chars), return_sequences=True),
        TimeDistributed(Dense(num_chars, activation='softmax'))
    ])
    
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary() 
    
    input_text = f.read()
    
    targets = input_text[1:]
    input_text = input_text[:(len(input_text) - 1)]
      
    oh_input_chars = bliteral_to_categorical(input_text)
    oh_targets = bliteral_to_categorical(targets)
    steps_per_epoch = oh_input_chars.shape[0] / batch_size
    
    train_generator = generate_sequences(oh_input_chars, oh_targets, seq_len=seq_len, batch_size=batch_size)
    
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
    checkpointer = ModelCheckpoint(filepath=model_save_path, verbose=1)
    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, callbacks=[print_callback, checkpointer])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 100, 15)           16320     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 100, 256)          4096      
Total params: 20,416
Trainable params: 20,416
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
 113/5524 [..............................] - ETA: 13:13 - loss: 4.4451 - acc: 0.1364